# Database Engineering

In [1]:
# Import dependencies
import pandas as pd
import csv
import numpy as np
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text, create_engine, MetaData, ForeignKey

Use Pandas to read your cleaned measurements and stations CSV data.

In [2]:
# Import Countries CSVs
df_countries = pd.read_csv("countries.csv")
df_countries.head()

,id,country,lat,lon
0,0,Italy,42.833333,12.833333
1,1,Portugal,39.500000,-8.000000
2,2,US,39.828175,-98.579500
3,3,Spain,40.000000,-4.000000
4,4,France,46.000000,2.000000


In [3]:
# Import Province CSV
df_provinces = pd.read_csv("provinces.csv")
df_provinces.head()

,id,province,pro_lat,pro_lon,country_id
0,0,Sicily & Sardinia,37.599994,14.015356,0
1,1,Douro,41.512019,-5.492255,1
2,2,Oregon,43.804133,-120.554201,2
3,3,Michigan,44.314844,-85.602364,2
4,4,Northern Spain,43.263588,-2.929068,3


In [4]:
# Import Wineries CSV
df_wineries = pd.read_csv("wineries.csv")
df_wineries.head()

,id,winery,country_id,province_id
0,0,Nicosia,0,0
1,1,Quinta dos Avidagos,1,1
2,2,Rainstorm,2,2
3,3,St. Julian,2,3
4,4,Sweet Cheeks,2,2


In [5]:
# Import Wines CSV
df_wines = pd.read_csv("wines.csv")
df_wines.head()

,id,title,variety,price,points,country_id,province_id,winery_id
0,0,Nicosia 2013 Vulkà Bianco (Etna),White Blend,0,87,0.0,0.0,0.0
1,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,15,87,1.0,1.0,1.0
2,2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,14,87,2.0,2.0,2.0
3,3,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,13,87,2.0,3.0,3.0
4,4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,65,87,2.0,2.0,2.0


Use the engine and connection string to create a database called hawaii.sqlite.

In [6]:
# Create Engine and Connect it
engine = create_engine("sqlite:///winetasting.sqlite")

conn = engine.connect()

# Use declarative_base and create ORM classes for each table.


You will need a class for Measurement and for Station.
Make sure to define your primary keys.

In [7]:
# Set dec_base, primary/foreign keys, and classes for two items
Base = declarative_base()

class Countries(Base):
    __tablename__ = 'countries'
    id = Column(Integer)
    country = Column(String(40), ForeignKey('provinces.country_id'), primary_key = True)
    lat = Column(Float)
    lon = Column(Float)
    
class Provinces(Base):
    __tablename__ = 'provinces'
    id = Column(Integer, primary_key = True)
    country_id = Column(String(20), ForeignKey('wineries.country_id'))
    province = Column(String (50))
    pro_lon = Column(Float)
    pro_lat = Column(Float)

class Wineries(Base):
    __tablename__ = 'wineries'
    id = Column(Integer, primary_key = True)
    country_id = Column(String(20), ForeignKey('wines.country_id'))
    province_id = Column(Float)
    winery = Column(String(40))
    
class Wines(Base):
    __tablename__ = 'wines'
    id = Column(Integer, primary_key = True)
    country_id = Column(String(20), ForeignKey('provinces.country_id'))
    province_id = Column(Float)
    winery_id = Column(Float)
    title = Column(String(255))
    variety = Column(String(40))
    points = Column(Float)
    price = Column(Float)

Once you have your ORM classes defined, create the tables in the database using 'create_all'.

In [8]:
# Create database with the newly created classes inside of it
Base.metadata.create_all(engine)

In [9]:
# Bring in CSV files and change them from simple pandas to records-type pandas
df_countries_clean = pd.read_csv('countries.csv')
df_provinces_clean = pd.read_csv('provinces.csv')
df_wineries_clean = pd.read_csv('wineries.csv')
df_wines_clean = pd.read_csv('wines.csv')


countries_data = df_countries_clean.to_dict(orient='records')
provinces_data = df_provinces_clean.to_dict(orient='records')
wineries_data = df_wineries_clean.to_dict(orient='records')
wines_data = df_wines_clean.to_dict(orient='records')
wines_data[0]

{'id': 0,
 'title': 'Nicosia 2013 Vulkà Bianco  (Etna)',
 'variety': 'White Blend',
 'price': 0,
 'points': 87,
 'country_id': 0.0,
 'province_id': 0.0,
 'winery_id': 0.0}

In [10]:
# Create metadata item to be used in creating the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Create tables
tab_countries = sqlalchemy.Table('countries', metadata, autoload=True)
tab_provinces = sqlalchemy.Table('provinces', metadata, autoload = True)
tab_wineries = sqlalchemy.Table('wineries', metadata, autoload=True)
tab_wines = sqlalchemy.Table('wines', metadata, autoload=True)

In [12]:
# Insert the panda records into the tables, column by column
conn.execute(tab_countries.insert(), countries_data)
conn.execute(tab_provinces.insert(), provinces_data)
conn.execute(tab_wineries.insert(), wineries_data)
conn.execute(tab_wines.insert(), wines_data)

In [13]:
# Verify that the info has been inserted into the table.
conn.execute("select * from countries limit 10").fetchall()

[(0, 'Italy', 42.833333, 12.833333),
 (1, 'Portugal', 39.5, -8.0),
 (2, 'US', 39.828175, -98.5795),
 (3, 'Spain', 40.0, -4.0),
 (4, 'France', 46.0, 2.0),
 (5, 'Germany', 51.5, 10.5),
 (6, 'Argentina', -34.0, -64.0),
 (7, 'Chile', -30.0, -71.0),
 (8, 'Australia', -25.0, 135.0),
 (9, 'Austria', 47.333333, 13.333332999999998)]

In [14]:
# Verify that the info has been inserted into the table.
conn.execute("select * from provinces limit 10").fetchall()

[(0, '0', 'Sicily & Sardinia', 14.0153557, 37.5999938),
 (1, '1', 'Douro', -5.4922547999999995, 41.51201939999999),
 (2, '2', 'Oregon', -120.55420120000001, 43.8041334),
 (3, '2', 'Michigan', -85.60236429999999, 44.3148443),
 (4, '3', 'Northern Spain', -2.9290679, 43.263588299999995),
 (5, '4', 'Alsace', 7.441624099999999, 48.3181795),
 (6, '5', 'Rheinhessen', 10.5, 51.5),
 (7, '2', 'California', -119.4179324, 36.778261),
 (8, '5', 'Mosel', 6.739917299999999, 49.1169679),
 (9, '6', 'Other', -64.0, -34.0)]

In [15]:
conn.execute("select * from wineries limit 10").fetchall()

[(0, '0', 0.0, 'Nicosia'),
 (1, '1', 1.0, 'Quinta dos Avidagos'),
 (2, '2', 2.0, 'Rainstorm'),
 (3, '2', 3.0, 'St. Julian'),
 (4, '2', 2.0, 'Sweet Cheeks'),
 (5, '3', 4.0, 'Tandem'),
 (6, '0', 0.0, 'Terre di Giurfo'),
 (7, '4', 5.0, 'Trimbach'),
 (8, '5', 6.0, 'Heinz Eifel'),
 (9, '4', 5.0, 'Jean-Baptiste Adam')]

In [16]:
conn.execute("select * from wines limit 10").fetchall()

[(0, '0.0', 0.0, 0.0, 'Nicosia 2013 Vulkà Bianco  (Etna)', 'White Blend', 87.0, 0.0),
 (1, '1.0', 1.0, 1.0, 'Quinta dos Avidagos 2011 Avidagos Red (Douro)', 'Portuguese Red', 87.0, 15.0),
 (2, '2.0', 2.0, 2.0, 'Rainstorm 2013 Pinot Gris (Willamette Valley)', 'Pinot Gris', 87.0, 14.0),
 (3, '2.0', 3.0, 3.0, 'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)', 'Riesling', 87.0, 13.0),
 (4, '2.0', 2.0, 2.0, "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)", 'Pinot Noir', 87.0, 65.0),
 (5, '3.0', 4.0, 4.0, 'Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra)', 'Tempranillo-Merlot', 87.0, 15.0),
 (6, '0.0', 0.0, 0.0, 'Terre di Giurfo 2013 Belsito Frappato (Vittoria)', 'Frappato', 87.0, 16.0),
 (7, '4.0', 5.0, 5.0, 'Trimbach 2012 Gewurztraminer (Alsace)', 'Gewürztraminer', 87.0, 24.0),
 (8, '5.0', 6.0, 6.0, 'Heinz Eifel 2013 Shine Gewürztraminer (Rheinhessen)', 'Gewürztraminer', 87.0, 12.0),
 (9, '4.0', 5.0, 5.0, 'Jean-Baptiste Adam 201